# <a id='toc1_'></a>[Mobile Robotics Project Report](#toc0_)

---
Marcus Cemes, Pable Paller, Adrien Pannatier,  Carolina Rodrigues Fidalgo


<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    This notebook describes the different parts of our project for the course micro-452: Basics of mobile robotics.<br/>
    In the following cells we will describe how the modules work and give a structure to run the project.
</div>

**Table of contents**<a id='toc0_'></a>    
- [Mobile Robotics Project Report](#toc1_)    
  - [Run requirements](#toc1_1_)    
  - [Add the modules to path](#toc1_2_)    
  - [Vision](#toc1_3_)    
  - [Filtering](#toc1_4_)    
    - [Prediction](#toc1_4_1_)    
    - [Update](#toc1_4_2_)    
  - [Global navigation](#toc1_5_)    
  - [Big Brain](#toc1_6_)    
  - [Motion control](#toc1_7_)    
  - [Local navigation](#toc1_8_)    
  - [Running the program as a whole](#toc1_9_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Run requirements](#toc0_)

All instructions to run the project are to be found in the [readme](README.md)


## <a id='toc1_2_'></a>[Add the modules to path](#toc0_)

The following trickery adds our Python module to the Jupyter environment.

In [ ]:
from sys import path
path.append("./app")
path.append("./report")

Let's also import some useful libraries for this notebook.

In [ ]:
import asyncio
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

One of the main structures of our project is the context. Created to be shared between the modules, the context holds the following data:<br/>

<span style="text-decoration:underline;font-weight:bold;">[Context Class](app/context.py)</span>
- <strong>node</strong>: node of the main thymio
- <strong>node_top</strong>: node of the top thymio
- <strong>pool</strong>: necessary part to allow multiple threading in python
- <strong>state</strong>: state containing all the necessary elements to describe the scene
- <strong>scene_update</strong>: signaling of scene update from vision
- <strong>pose_update</strong>: signaling position update from filtering
- <strong>debug_update</strong>: boolean to allow debugging vision display in the ui

Let us create this context

In [ ]:
from app.config import *
from app.context import Context
from app.state import State
from report.mock_pool import MockPool
import report.map
import report.report_functions 

ctx = Context(node=None, node_top=None, pool=MockPool(), state=State())

ctx.state.physical_size = PHYSICAL_SIZE_CM
ctx.state.subdivisions = SUBDIVISIONS

The [State Class](app/state.py) holds the necessary information on the thymio to allow the correct progression of the robot.</br>
State also contains the elements needed by the ui interface to display the scene. 

In order to visualise the state of the map without the use of the Web UI, we've written a `plot_map(ctx)` function that uses the matplotlib library.

In [ ]:
report.map.plot_map(ctx, "map of the scene")

We can now import modules and show how they work!

___
## <a id='toc1_3_'></a>[Vision](#toc0_)

In [ ]:
from app.vision import Vision

In [ ]:
vision = Vision(ctx, external=False, live=False, image_path="assets/test_frame_01.jpg")
report.map.reset(ctx)

In [ ]:
vision.calibrate()

In [ ]:
obs = vision.next()

In [ ]:
vision_pose = (obs.back[0], obs.back[1], report.report_functions.angle(obs.back, obs.front))
print(vision_pose)

display the vision retranscription

In [ ]:
ctx.state.position = (obs.back[0], obs.back[1])
ctx.state.orientation = report.report_functions.angle(obs.back, obs.front)
ctx.state.obstacles = obs.obstacles

In [ ]:
report.map.plot_map(ctx, "vision retranscription")

In [ ]:
report.map.show_picture("assets/test_frame_01.jpg")

In [ ]:
report.map.plot_map(ctx, "Vision Map")

## <a id='toc1_4_'></a>[Filtering](#toc0_)

Another important module of our project is filtering. Filtering is taking the information given by sensors and the camera to be able to guess where the thymio is on the map and where it's going.</br>
For this project we decieded to use an [Extanded Kalman Filter](app/EXF.py) and an intermediate module called [filtering](app/filtering.py) to comunicate with the main module.</br>
The EKF was chosen for its capability of predicting a non-linear evolution of the state and also to be able to dissociate information coming from the motors and information given by the camera.</br>
For this purpose, two functions where used: [predict](#toc1_4_1_) and [update](#toc1_4_2_). We will come back to these parts in the subsections below.
___


first let's import the module 

In [ ]:
from app.filtering import Filtering
from asyncio import sleep

In [ ]:
#run this cell to reset this section
#creation of the filter
filtering = Filtering(ctx)
report.map.reset(ctx)
report.report_functions.start_movement_simulation(filtering)

### <a id='toc1_4_1_'></a>[Prediction](#toc0_)

A specificity of our project is the time delay between camera and motor sensors. Since the motor sensors response is faster than the one from vision, what we do is predict the position of the robot with the motor sensors at a greater frequency than the update with vision information. The time interval between each prediction is dynamic and adapts to correctly predict each time the motor sensors send variables.

The next position and orientation are then predicted.

Let us run it to see how it works
___

Let's take a look at our current position.

In [ ]:
report.report_functions.print_pose(ctx)

In [ ]:
report.map.plot_map(ctx, "Filtering Map")

To execute a filtering step let's make the Thymio move forward:

In [ ]:
filtering.process_event({"motor.left.speed": [100], "motor.right.speed": [100]})
report.report_functions.stop_movement_simulation(filtering)

The state should now be updated

In [ ]:
report.report_functions.print_pose(ctx)
report.map.plot_map(ctx, "Filtering Map")

Let's do it again, turning the robot this time

In [ ]:
filtering.process_event({"motor.left.speed": [50], "motor.right.speed": [-50]})
report.report_functions.stop_movement_simulation(filtering)

In [ ]:
report.report_functions.print_pose(ctx)
report.map.plot_map(ctx, "Filtering Map")

we see that filtering is correctly predicting the progression of the simulated robot

### <a id='toc1_4_2_'></a>[Update](#toc0_)

To reduce the error caused by motors, vision must be used as much as possible. If vision is lost for a while, the update step brings back the Thymio to it's correct position taking into consideration the computed prediction since it's disapearing.
___

now to display the update function, we use the image and position of the robot given by the [Vision module](#toc1_3_)   

The position after our prediction step is:

In [ ]:
report.report_functions.print_pose(ctx)

now we update the position with the one given by vision

In [ ]:
vision_position = vision #get image from vision
filtering.update(vision_pose)

after the vision update, the new position is:

In [ ]:
report.report_functions.print_pose(ctx)
report.map.plot_map(ctx, "Filtering Map")

## <a id='toc1_5_'></a>[Global navigation](#toc0_)

In [ ]:
import app.global_navigation

global_nav = app.global_navigation.GlobalNavigation(ctx)

# Reset the state related to path finding.
# You can rerun this cell to reset this section.
report.map.reset(ctx)

At the moment our map is completely empty. In order to invoke our pathfinding algorithm, we need to set the <font color='red'>Thymio's position</font>  and a desired <font color='cyan'>end point</font>.

In [ ]:
# Update the state
ctx.state.position = (10, 10)
ctx.state.end = (90, 110)

# Recompute the path
await global_nav._recompute_path()

# Plot the result
report.map.plot_map(ctx, "Path-finding map")

**🎉 We have a path!**

Let's not celebrate yet, we can make this much more interesting. Our pathfinding uses an implementation of the [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm). While this may not be the most fastest or most efficient algorithm, it _does_ guarantee optimal results for a given graph.

Our node graph is a grid of equaly spaced cells, where edges are made between directly adjacent cells and diagonal cells, with a cost of 1 and $\sqrt{2}$, respectively. This allows our robot to also travel diagonally and avoid grid-like movement.

<div align="center" style="padding: 1rem 0;">
    <figure>
        <img src="assets/report/dijkstra-nodes.png" style="width: 8rem;" />
        <figcaption style="font-style: italic;">Edges for a graph node to its adjacent nodes.</figcaption>
    </figure>
</div>

Ideally, we would like all nodes to be connected to every other node, given that there is a line-of-sight between them, however this comes with a huge computational burden. This simple implementation (mixed with our path optimisation algorithm covered later) provides adequate results in roughly 50 ms for 64 subdivisions.

Time for some obstacles! Obstacles come directly from the vision module as a matrix of 8-bit integers, where any value other an 0 represent an occupied cell. The size of the matrix coincides with the node graph, an occupied cell will mark a node as unvisitable.

**Note:** While we could have also used a matrix of booleans, integers are slightly easier to work with and compose with other functions such as convolutions. Additionally, it has a smaller JSON serialisation footprint for sending over WebSockets to the Web UI.

In [ ]:
# Reset the matrix
ctx.state.obstacles[:,:] = 0

# Add two obstacles
ctx.state.obstacles[30:45, 40:55] = 1
ctx.state.obstacles[20:30, 5: 20] = 1

# Recompute the path
await global_nav._recompute_path()

# Plot the result
report.map.plot_map(ctx, "Path-finding map")

The dark-blue region represents the obstacles that we set. The cyan regions are the obstacle boundaries that are generated using a convolution of a circular kernel matrix.

In [ ]:
kernel = global_nav._safety_margin_kernel()

report.map.plot_image(kernel, "Obstacle boundary convolution kernel", colourbar=True)

The boundary is equaly treated as unvisitable by the pathfinding algorith. This ensures that the Thymio does not hit any objects with its wheels as it's navigating the path. It a gap is too small, the Thymio will not try to fit through it.

In [ ]:
# Reset the matrix
ctx.state.obstacles[:,:] = 0

# Add two obstacles with a gap
ctx.state.obstacles[31:33, 15:29] = 1
ctx.state.obstacles[31:33, 42:50] = 1

# Recompute the path
await global_nav._recompute_path()

# Plot the result
report.map.plot_map(ctx, "Path-finding map")

### Path optimisation

Our chosen node graph constraints our robots movements more or less to a grid, with the ability of also being able to move diagonally. This creates a lot of path waypoints, but also creates less natural-like movement.

To combat this, we have an additional post-pathfinding step to iteratively try to reduce the path to as few waypoints as necessary that have free line of sight between them. This does not find the most optimal solution, but the results are more than adequate.

The algorithm goes as follows:

1. For $i = 2,...,M-1$, where $M$ is the number of waypoints
2. Check whether there is a free line-of-sight between waypoints $w_{i-1}$ and $w_{i+1}$
3. If yes, remove the waypoint $w_i$, otherwise consider the next waypoint $w_{i+1}$

In order to calculate free line-of-sight, it's necessary to enumerate all map cells that the segment between $w_{i-1}$ and $w_{i+1}$ travels through to check their occupation status. This is done using an algorithm such as [Bresenham's line algorithm](https://en.wikipedia.org/wiki/Bresenham%27s_line_algorithm). We found an improved version that also uses integer-only math [here](https://playtechs.blogspot.com/2007/03/raytracing-on-grid.html).

In [ ]:
from app.path_finding.path_optimiser import raytrace

p1 = (1, 2)
p2 = (8, 6)

matrix = np.zeros((10, 10))

for (i, j) in raytrace(p1, p2):
    matrix[j][i] = 1

report.map.plot_raytrace(matrix, p1, p2)

Let's put this into practice using the path that we found previously.

In [ ]:
from app.path_finding.path_optimiser import PathOptimiser

map = report.map.create_map(ctx)

optimiser = PathOptimiser(map)

# The output of the pathfinding algorithm is in physical space [cm],
# it needs to be converted back into map indicies for this demonstration.
# Usually the path optimisation is done as an intermediate step by the
# path finding algorithm, before the conversion to physical space.
path = report.map.path_to_coords(ctx.state.path)

# The optimiser mutates the input list directly
optimised_path = optimiser.optimise(path.copy())

report.map.plot_path_optimisation(path, optimised_path, map)

Path optimisation can be toggled dynamically by setting the `ctx.state.optimise` boolean flag.

In [ ]:
# Enable path optimisation
ctx.state.optimise = True

# Reset the matrix
ctx.state.obstacles[:,:] = 0

# Add two obstacles
ctx.state.obstacles[30:45, 40:55] = 1
ctx.state.obstacles[20:30, 5: 20] = 1

await global_nav._recompute_path()

report.map.plot_map(ctx, "Path-finding map")

## <a id='toc1_6_'></a>[Big Brain](#toc0_)

## <a id='toc1_7_'></a>[Motion control](#toc0_)

## <a id='toc1_8_'></a>[Local navigation](#toc0_)

## <a id='toc1_9_'></a>[Running the program as a whole](#toc0_)

now that we went through all the main modules we can run the app containing the project

In [ ]:
!python -m app